In [1]:
import findspark
findspark.init('/home/ubuntu/spark-3.2.1-bin-hadoop2.7')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('DM-LR').getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/12 02:22:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# Load data. 
data = spark.read.csv('BDAS_Iteration_Dataset/final_df.csv',inferSchema=True,header=True)

# Load Trained Model Logistic Regression
from pyspark.ml.classification import LogisticRegressionModel
trained_LR_model = LogisticRegressionModel.load("LogisticRegression_model")

In [3]:
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline

# Create StringIndexers for categorical predictor variables
gender_indexer = StringIndexer(inputCol="gender", outputCol="gender_index")
marital_indexer = StringIndexer(inputCol="marital", outputCol="marital_index")
parenthood_indexer = StringIndexer(inputCol="parenthood", outputCol="parenthood_index")

# Create a StringIndexer for the target variable
target_indexer = StringIndexer(inputCol="predicted_category", outputCol="label")

# Assemble the predictor variables into a single features column
assembler = VectorAssembler(
    inputCols=["gender_index", "marital_index", "parenthood_index"],
    outputCol="features"
)

# Define a pipeline to execute the transformations
pipeline = Pipeline(stages=[gender_indexer, marital_indexer, parenthood_indexer, target_indexer, assembler])

# Fit and transform the data using the pipeline
model = pipeline.fit(data)
data = model.transform(data)

# Select the relevant columns for your machine learning model
final_data = data.select("label", "features")

# Show the first few rows of the final_data DataFrame
final_data.show()

+-----+-------------+
|label|     features|
+-----+-------------+
|  0.0|[0.0,1.0,1.0]|
|  0.0|[0.0,1.0,1.0]|
|  0.0|    (3,[],[])|
|  0.0|[0.0,1.0,1.0]|
|  0.0|[0.0,1.0,1.0]|
|  0.0|[0.0,1.0,1.0]|
|  0.0|[0.0,1.0,1.0]|
|  0.0|    (3,[],[])|
|  0.0|    (3,[],[])|
|  0.0|[1.0,0.0,0.0]|
|  0.0|[0.0,1.0,1.0]|
|  0.0|[0.0,1.0,1.0]|
|  0.0|[0.0,1.0,1.0]|
|  0.0|    (3,[],[])|
|  0.0|    (3,[],[])|
|  0.0|    (3,[],[])|
|  0.0|[0.0,1.0,0.0]|
|  0.0|[0.0,1.0,1.0]|
|  0.0|[0.0,0.0,1.0]|
|  0.0|    (3,[],[])|
+-----+-------------+
only showing top 20 rows



In [5]:
import pandas as pd

# Access the coefficients from the loaded model
coefficients = trained_LR_model.coefficients
intercept = trained_LR_model.intercept

# Assuming you have a list of feature names
feature_names = ["gender_index", "marital_index", "parenthood_index"]

# Create a Pandas DataFrame for coefficients and feature names
coefficients_df = pd.DataFrame({
    'Feature': feature_names,
    'Coefficient': coefficients
})

# Add an intercept row for the intercept term
intercept_row = pd.DataFrame({'Feature': ['Intercept'], 'Coefficient': [intercept]})
coefficients_df = coefficients_df.append(intercept_row)

# Display the coefficients DataFrame
print(coefficients_df)


            Feature  Coefficient
0      gender_index     0.560120
1     marital_index     0.443305
2  parenthood_index     0.355650
0         Intercept    -2.036483


/tmp/ipykernel_18123/3331258908.py:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  coefficients_df = coefficients_df.append(intercept_row)
